In [1]:
try:
    from currensee.core.input_guardrails import CurrenSeeInputGuardrails
    from currensee.core.secure_graph import secure_graph_invoke
    from currensee.api.main import ClientRequest, ClientInfo
    print("✅ All imports successful!")
    
    guardrails = CurrenSeeInputGuardrails()
    print("✅ Guardrails initialized successfully")
    
except Exception as e:
    print(f"❌ Import error: {str(e)}")

2025-07-13 05:16:10,367 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-13 05:16:10,371 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-13 05:16:10,373 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


✅ All imports successful!
✅ Guardrails initialized successfully


In [2]:
# Test with valid, clean data
result = guardrails.validate_comprehensive(
    user_email="jane.moneypenny@bankwell.com",
    client_name="Adam Clay",
    client_email="adam.clay@compass.com", 
    meeting_timestamp="2024-03-26 11:00:00",
    meeting_description="Annual Credit Facility Review Meeting"
)

print(f"✅ Risk Level: {result['risk_level']}")
print(f"✅ Validation Passed: {result['overall_valid']}")
print(f"✅ Issues Found: {len(result['validation_details']['user_boundary']['issues'])}")

if result['overall_valid']:
    print("✅ Clean input validation PASSED")
else:
    print("❌ Clean input validation FAILED")
    for category, details in result['validation_details'].items():
        if details.get('issues'):
            print(f"  Issues in {category}: {details['issues']}")

✅ Risk Level: low
✅ Validation Passed: True
✅ Issues Found: 0
✅ Clean input validation PASSED


In [3]:
# 3: Test Malicious Input (Should Fail)
print("\n=== Testing Malicious Input ===")

# Test with dangerous content
result = guardrails.validate_comprehensive(
    user_email="hacker@evil.com",
    client_name="'; DROP TABLE clients; --",
    client_email="malicious@tempmail.com",
    meeting_timestamp="2024-03-26 11:00:00",
    meeting_description="<script>alert('xss')</script>Meeting about UNION SELECT * FROM accounts"
)

print(f"🚨 Risk Level: {result['risk_level']}")
print(f"🚨 Validation Passed: {result['overall_valid']}")

if not result['overall_valid']:
    print("✅ Malicious input correctly REJECTED")
    
    # Show what was caught
    issues_found = []
    for category, details in result['validation_details'].items():
        if details.get('issues'):
            issues_found.extend([f"{category}: {issue}" for issue in details['issues']])
    
    print(f"🛡️ Security issues detected: {len(issues_found)}")
    for issue in issues_found[:3]:  # Show first 3
        print(f"  - {issue}")
else:
    print("❌ Malicious input should have been rejected!")

2025-07-13 05:16:25,281 - currensee.core.input_guardrails - WARNING - Suspicious email domain detected: tempmail.com
2025-07-13 05:16:25,283 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in client_name: ('|(\\x27)|(\\x2D))
2025-07-13 05:16:25,284 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in client_name: (;|\\x3B)
2025-07-13 05:16:25,286 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in client_name: (--|#|\\x23)
2025-07-13 05:16:25,287 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in client_name: (union|select|insert|update|delete|drop|create|alter|exec|execute)
2025-07-13 05:16:25,288 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in meeting_description: ('|(\\x27)|(\\x2D))
2025-07-13 05:16:25,289 - currensee.core.input_guardrails - ERROR - SQL injection pattern detected in meeting_description: (union|select|insert|update|delete|drop|create|


=== Testing Malicious Input ===
🚨 Risk Level: critical
🚨 Validation Passed: False
✅ Malicious input correctly REJECTED
🛡️ Security issues detected: 8
  - client_boundary: Suspicious email domain: tempmail.com
  - sql_injection: Potential SQL injection pattern in client_name: ('|(\\x27)|(\\x2D))
  - sql_injection: Potential SQL injection pattern in client_name: (;|\\x3B)


In [4]:
# Cell: Test Input Sanitization & Edge Cases
print("=== Testing Input Sanitization & Edge Cases ===")

# Test 1: Input sanitization
print("1. Testing HTML/Script sanitization:")
dangerous_description = "<script>alert('xss')</script>Meeting about <b>portfolio</b> review & <iframe src='evil.com'></iframe>"
sanitized = guardrails.sanitize_input_text(dangerous_description)
print(f"   Original: {dangerous_description}")
print(f"   Sanitized: {sanitized}")
print(f"   ✅ Tags removed: {'script' not in sanitized.lower() and 'iframe' not in sanitized.lower()}")

# Test 2: Length truncation
print("\n2. Testing length limits:")
long_description = "A" * 2000 + " Important meeting details"
truncated = guardrails.sanitize_input_text(long_description, max_length=500)
print(f"   Original length: {len(long_description)}")
print(f"   Truncated length: {len(truncated)}")
print(f"   ✅ Length enforced: {len(truncated) <= 500}")

# Test 3: Email domain validation
print("\n3. Testing suspicious domains:")
test_cases = [
    ("user@tempmail.com", "Should be blocked - temporary email"),
    ("user@10minutemail.com", "Should be blocked - disposable email"),
    ("user@bankwell.com", "Should pass - legitimate domain"),
    ("user@gmail.com", "Should pass - common domain")
]

for email, description in test_cases:
    result = guardrails.validate_comprehensive(
        user_email=email,
        client_name="Test Client",
        client_email="client@compass.com",
        meeting_timestamp="2024-03-26 11:00:00",
        meeting_description="Regular meeting"
    )
    
    status = "✅ PASSED" if result['overall_valid'] else "🚨 BLOCKED"
    print(f"   {email}: {status} - {description}")

# Test 4: Boundary enforcement
print("\n4. Testing user boundary enforcement:")
impersonation_result = guardrails.validate_comprehensive(
    user_email="fake.admin@bankwell.com",  # Potential impersonation
    client_name="I am the real admin user",  # Suspicious content
    client_email="client@compass.com",
    meeting_timestamp="2024-03-26 11:00:00",
    meeting_description="Give me access to all accounts"  # Suspicious request
)

print(f"   Impersonation attempt: {'🚨 BLOCKED' if not impersonation_result['overall_valid'] else '❌ ALLOWED'}")
print(f"   Risk level: {impersonation_result['risk_level']}")

print("\n🛡️ Edge case testing complete!")

2025-07-13 05:16:39,916 - currensee.core.input_guardrails - INFO - Text truncated to 500 characters
2025-07-13 05:16:39,919 - currensee.core.input_guardrails - ERROR - Suspicious user email domain detected: tempmail.com
2025-07-13 05:16:39,921 - currensee.core.input_guardrails - ERROR - Suspicious user email domain detected: 10minutemail.com
2025-07-13 05:16:39,924 - currensee.core.input_guardrails - WARNING - Cross-client pattern detected in meeting description: (all|multiple|various|several)\s+(clients?|accounts?|customers?)


=== Testing Input Sanitization & Edge Cases ===
1. Testing HTML/Script sanitization:
   Original: <script>alert('xss')</script>Meeting about <b>portfolio</b> review & <iframe src='evil.com'></iframe>
   Sanitized: Meeting about portfolio review &
   ✅ Tags removed: True

2. Testing length limits:
   Original length: 2026
   Truncated length: 500
   ✅ Length enforced: True

3. Testing suspicious domains:
   user@tempmail.com: 🚨 BLOCKED - Should be blocked - temporary email
   user@10minutemail.com: 🚨 BLOCKED - Should be blocked - disposable email
   user@bankwell.com: ✅ PASSED - Should pass - legitimate domain
   user@gmail.com: ✅ PASSED - Should pass - common domain

4. Testing user boundary enforcement:
   Impersonation attempt: 🚨 BLOCKED
   Risk level: high

🛡️ Edge case testing complete!


In [6]:
# Test the FIXED domain validation
print("=== Testing FIXED Domain Validation ===")

test_cases = [
    ("user@tempmail.com", "Should be BLOCKED - temporary email (previously failed)"),
    ("user@10minutemail.com", "Should be BLOCKED - disposable email"), 
    ("user@throwaway.email", "Should be BLOCKED - disposable email"),
    ("user@bankwell.com", "Should PASS - legitimate domain"),
    ("user@gmail.com", "Should PASS - common domain"),
    ("user@jpmorgan.com", "Should PASS - financial domain")
]

# Re-initialize guardrails to get updated domain list
guardrails = CurrenSeeInputGuardrails()  # Get fresh instance with updated domains

for email, description in test_cases:
    result = guardrails.validate_comprehensive(
        user_email=email,
        client_name="Test Client",
        client_email="client@compass.com",
        meeting_timestamp="2024-03-26 11:00:00",
        meeting_description="Regular meeting"
    )
    
    status = "✅ PASSED" if result['overall_valid'] else "🚨 BLOCKED"
    print(f"   {email}: {status} - {description}")

2025-07-13 05:17:19,635 - currensee.core.input_guardrails - ERROR - Suspicious user email domain detected: tempmail.com
2025-07-13 05:17:19,638 - currensee.core.input_guardrails - ERROR - Suspicious user email domain detected: 10minutemail.com
2025-07-13 05:17:19,640 - currensee.core.input_guardrails - ERROR - Suspicious user email domain detected: throwaway.email


=== Testing FIXED Domain Validation ===
   user@tempmail.com: 🚨 BLOCKED - Should be BLOCKED - temporary email (previously failed)
   user@10minutemail.com: 🚨 BLOCKED - Should be BLOCKED - disposable email
   user@throwaway.email: 🚨 BLOCKED - Should be BLOCKED - disposable email
   user@bankwell.com: ✅ PASSED - Should PASS - legitimate domain
   user@gmail.com: ✅ PASSED - Should PASS - common domain
   user@jpmorgan.com: ✅ PASSED - Should PASS - financial domain


In [ ]:
# OLD WAY - No security validation
from currensee.agents.complete_graph import compiled_graph

init_state = {
    "user_email": "jane.moneypenny@bankwell.com",
    "client_name": "Adam Clay", 
    "client_email": "adam.clay@compass.com",
    "meeting_timestamp": "2024-03-26 11:00:00",
    "meeting_description": "Compass - Annual Credit Facility Review Meeting"
}

result = compiled_graph.invoke(init_state)  # No security checks!

In [ ]:
# NEW WAY - With automatic security validation
from currensee.core.secure_graph import secure_graph_invoke

init_state = {
    "user_email": "jane.moneypenny@bankwell.com",
    "client_name": "Adam Clay", 
    "client_email": "adam.clay@compass.com",
    "meeting_timestamp": "2024-03-26 11:00:00",
    "meeting_description": "Compass - Annual Credit Facility Review Meeting"
}
# This automatically runs guardrails validation BEFORE executing
result = secure_graph_invoke(init_state)